In [ ]:
# -------------------------------
# Multi-seed, multi-class evaluation with .npz export (CIFAR-100)
# -------------------------------

import os
import numpy as np
import torch
import torch.nn as nn
from lib.cifar100_utils import (
    build_cifar_resnet, build_wideresnet, get_dataloaders, load_model, evaluate_model
)

# -------------------------------
# Configuration
# -------------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 512
NUM_CLASSES = 100
MODEL_DIR = "./../models/cifar100/"
DATASET_DIR = "./../data/"
RESULTS_DIR = "./../results/cifar100/"
os.makedirs(RESULTS_DIR, exist_ok=True)

# Seeds and removed classes to evaluate
SEEDS = [42, 602, 311, 637, 800, 543, 969, 122, 336, 93]   # example seeds
#SEEDS = [311, 637, 969]   # example seeds
REMOVE_CLASSES = [
    None, 4, 7, 6, 11, 14, 15, 19, 24, 23, 25, 29, 30, 
    31, 36, 35, 39, 44, 41, 49, 48, 51, 53, 59, 57, 60, 61, 
    68, 67, 72, 71, 76, 77, 80, 84, 88, 85, 92, 94, 99, 97] # CIFAR-100
#REMOVE_CLASSES = [None, 14, 23, 35, 49, 53, 61, 68, 72, 88, 97]

EPOCHS = 50
#EPOCHS = 200

# -------------------------------
# Model type
# -------------------------------
MODEL_TYPE = "resnet18"  # "resnet18" or "wideresnet"

# -------------------------------
# Run evaluation and save results
# -------------------------------
for seed in SEEDS:
    for remove_class in REMOVE_CLASSES:
        print(f"\n=== Evaluating: seed={seed}, remove_class={remove_class} ===")

        # Test data loader
        _, testloader = get_dataloaders(
            batch_size=BATCH_SIZE,
            remove_class=remove_class,
            dataset_dir=DATASET_DIR
        )

        # Model path
        model_name = f"cifar100_{MODEL_TYPE}_s{seed}_e{EPOCHS}"
        if remove_class is not None:
            model_name += f"_r{remove_class}"
        model_path = os.path.join(MODEL_DIR, model_name + ".pth")

        # Result path
        result_path = os.path.join(RESULTS_DIR, model_name + ".npz")

        # Skip if already exists
        if os.path.exists(result_path):
            print(f"Already exists: {result_path}")
            continue

        # Build model
        if MODEL_TYPE == "resnet18":
            model = build_cifar_resnet(num_classes=NUM_CLASSES, device=DEVICE)
        elif MODEL_TYPE == "wideresnet":
            model = build_wideresnet(num_classes=NUM_CLASSES, device=DEVICE)
        else:
            raise ValueError(f"Unknown model type: {MODEL_TYPE}")

        # Load pretrained weights
        load_model(model, model_path, device=DEVICE)

        # Evaluate
        print(f"Evaluating model: {model_path}")
        preds, targets, confs = evaluate_model(
            model, testloader, nn.CrossEntropyLoss(), device=DEVICE
        )

        # Save to npz
        np.savez_compressed(
            result_path,
            preds=preds,
            targets=targets,
            confs=confs
        )
        print(f"Saved results to {result_path}")